In this assignement, feel free to use the `sparse` module from `scipy`.

Use the cell below for your imports.

In [1]:
from scipy import sparse

implement the function `mat_mul_coo` that takes two sparse matrices in `coo` and returns their product.

In [21]:
def mat_mul_coo(a, b):
    a_coo = a.tocoo()
    b_coo = b.tocoo()
    if a_coo.shape[1] != b_coo.shape[0]:
        raise ValueError("Matrices cannot be multiplied")

    rows_a, cols_a, data_a = a_coo.row, a_coo.col, a_coo.data
    rows_b, cols_b, data_b = b_coo.row, b_coo.col, b_coo.data

    cols = np.unique(np.concatenate((cols_a, cols_b)))
    rows = np.unique(np.concatenate((rows_a, rows_b)))

    n_rows, n_cols = len(rows), len(cols)

    col_index = dict(zip(cols, range(n_cols)))
    row_index = dict(zip(rows, range(n_rows)))

    data = np.zeros((n_rows, n_cols))

    for r, c, d in zip(rows_a, cols_a, data_a):
        data[row_index[r], col_index[c]] += d

    for r, c, d in zip(rows_b, cols_b, data_b):
        data[row_index[r], col_index[c]] += d

    return coo_matrix(data)


In [22]:
result = mat_mul_coo(a, b)
expected = coo_matrix([[1, 2, 0], [0, 3, 4], [5, 0, 6]])
print(result.toarray())
print(expected.toarray())


[[2. 2. 0.]
 [0. 4. 4.]
 [5. 0. 7.]]
[[1 2 0]
 [0 3 4]
 [5 0 6]]


implement the function `mat_mul_csr` that takes two sparse matrices in `csr` format and returns their product.

In [3]:
from scipy.sparse import csr_matrix

def mat_mul_csr(a: csr_matrix, b: csr_matrix) -> csr_matrix:
    if a.shape[1] != b.shape[0]:
        raise ValueError("Incompatible shapes")
    b_transpose = b.transpose()
    result = csr_matrix((a.shape[0], b.shape[1]), dtype=a.dtype)
    for i in range(a.shape[0]):
        row_start = a.indptr[i]
        row_end = a.indptr[i+1]
        for j in range(b_transpose.shape[0]):
            col_start = b_transpose.indptr[j]
            col_end = b_transpose.indptr[j+1]
            dot_product = 0
            k1, k2 = row_start, col_start
            while k1 < row_end and k2 < col_end:
                if a.indices[k1] < b_transpose.indices[k2]:
                    k1 += 1
                elif a.indices[k1] > b_transpose.indices[k2]:
                    k2 += 1
                else:
                    dot_product += a.data[k1] * b_transpose.data[k2]
                    k1 += 1
                    k2 += 1
            if dot_product != 0:
                result[i, j] = dot_product
    return result


In [6]:
from scipy.sparse import csr_matrix

# Define the matrices A and B in csr format
A = csr_matrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
B = csr_matrix([[10, 11], [12, 13], [14, 15]])

# Compute the product AB using mat_mul_csr
AB = mat_mul_csr(A, B)

# Print the result
print(AB.toarray())


[[ 32  38]
 [ 95 113]
 [158 188]]


/usr/local/lib/python3.9/dist-packages/scipy/sparse/_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


implement a function `solve_lin_sys` that takes a matrix `A` in `csr` format and a vector `b` as a numpy array and solves the system `Ax = b`.

In [4]:
from typing import Union
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve

def solve_lin_sys(A: csr_matrix, b: Union[np.ndarray, list]) -> np.ndarray:
    x = spsolve(A, b)
    return x


In [5]:
from scipy.sparse import csr_matrix
import numpy as np

# Define the matrix A in csr format
A = csr_matrix([[3, 0, 2], [0, 1, 0], [1, 0, 4]])

# Define the vector b
b = np.array([1, 2, 3])

# Solve the system Ax = b
x = solve_lin_sys(A, b)

# Print the solution vector
print(x)


[-0.2  2.   0.8]
